# Python Insights - Analisando Dados com Python

### Case - Cancelamento de Clientes

Você foi contratado por uma empresa com mais de 800 mil clientes para um projeto de Dados. Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.

Base de dados e arquivos: https://drive.google.com/drive/folders/1uDesZePdkhiraJmiyeZ-w5tfc8XsNYFZ?usp=drive_link

In [1]:
# PASSO A PASSO
# PASSo 1: Importar a base de dados
# Passo 2: Visualizar a base de dados
# Passo 3: Corrigir a base de dados
# Passo 4: Primeira analise de cancelamento
# Passo 5: Analise da causa do cancelamento dos clientes

In [35]:
import pandas as pd

tabela = pd.read_csv("cancelamentos_sample.csv")
tabela = tabela.drop(columns="CustomerID")
display(tabela)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,23.0,Male,13.0,22.0,2.0,1.0,Standard,Annual,909.58,23.0,0.0
1,49.0,Male,55.0,16.0,3.0,6.0,Premium,Monthly,207.00,29.0,1.0
2,30.0,Male,7.0,1.0,0.0,8.0,Basic,Annual,768.78,7.0,0.0
3,26.0,Male,40.0,5.0,3.0,8.0,Premium,Annual,398.00,12.0,1.0
4,27.0,Female,17.0,30.0,5.0,6.0,Basic,Annual,507.00,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
49995,62.0,Female,35.0,7.0,2.0,8.0,Basic,Annual,232.00,15.0,1.0
49996,36.0,Male,43.0,21.0,2.0,30.0,Basic,Quarterly,928.00,30.0,1.0
49997,55.0,Male,42.0,8.0,1.0,12.0,Basic,Monthly,326.00,27.0,1.0
49998,40.0,Female,14.0,19.0,1.0,17.0,Premium,Quarterly,826.76,12.0,0.0


In [27]:
# PASSO 3: Corrigir a base de dados
# Valores vazios
tabela = tabela.dropna() 
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49996 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   idade                   49996 non-null  float64
 1   sexo                    49996 non-null  object 
 2   tempo_como_cliente      49996 non-null  float64
 3   frequencia_uso          49996 non-null  float64
 4   ligacoes_callcenter     49996 non-null  float64
 5   dias_atraso             49996 non-null  float64
 6   assinatura              49996 non-null  object 
 7   duracao_contrato        49996 non-null  object 
 8   total_gasto             49996 non-null  float64
 9   meses_ultima_interacao  49996 non-null  float64
 10  cancelou                49996 non-null  float64
dtypes: float64(8), object(3)
memory usage: 4.6+ MB


None

In [28]:
# PASSO 4: Primeira analise de cancelamento dos clientes (Qual % de cancelamento)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))


1.0    28393
0.0    21603
Name: cancelou, dtype: int64

1.0    56.8%
0.0    43.2%
Name: cancelou, dtype: object

In [29]:
display(tabela["duracao_contrato"].value_counts())
display(tabela["duracao_contrato"].value_counts(normalize=True).map("{:.1%}".format))

Annual       20156
Quarterly    19956
Monthly       9884
Name: duracao_contrato, dtype: int64

Annual       40.3%
Quarterly    39.9%
Monthly      19.8%
Name: duracao_contrato, dtype: object

In [30]:
# agrupamento
# duracao_contrato - cancelou
# anual - 50%
# trimestral - 35%
# mensal - 90%

agrupamento = tabela.groupby("duracao_contrato").mean(numeric_only=True)
display(agrupamento)


,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
duracao_contrato,,,,,,,,
Annual,38.783985,31.416452,15.910449,3.277585,12.533985,650.925840,14.231544,0.464080
Monthly,41.428470,30.677964,15.577600,4.923917,15.078814,547.508921,15.392655,1.000000
Quarterly,38.833935,31.522099,15.842504,3.256113,12.480006,654.102443,14.364602,0.458759


In [36]:
# Todos os clientes do mensal cancelaram
# Sugestão: Oferecer descontos nos contratos anuais/semestrais

tabela = tabela[tabela["duracao_contrato"] != "Monthly"]
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))


0.0    21606
1.0    18510
Name: cancelou, dtype: int64

0.0    53.9%
1.0    46.1%
Name: cancelou, dtype: object

In [37]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable


In [38]:
# PASSO 5: Analise a causa do cancelamento dos clientes
# Criar graficos para fazer analise

import plotly.express as px

#criar o grafico
grafico = px.histogram(tabela, x="idade", color="cancelou")
#exibir o grafico
grafico.show()